# **MACHINE LEARNING NANODEGREE**
***
## **PROJETO FINAL**
***

## **Análise Preditiva de Cota para Exercício Parlamentar (CEAP)**

## ** PARTE 3 - Predição de Dados**
***

## ** DEPUTADO ROCHA**

***

## ** INTRODUÇÃO **

Realizaremos uma análise preditiva dos gastos para o ano de 2018 utilizando-se de algoritmo de aprendizado supervisionado.

É importante ressaltar que um deputado, salvo reeleição, terá registro de apenas quatros anos de despezas da CEAP. O que acaba sendo relativamente poucos dados os três primeiros anos para previsão dos gastos do último ano do mandato. Esta característica do dataset pode causar overfitting e ter impactos negativos com outliers.

Para contornar este problema devemos buscar utilizar algoritmos de menor complexidade, com menor número de parâmetros.
Tendo em vista estes desafios iremos utilizar dois tipos de lagoritmo:
> 1. LINEAR REGRESSION
> 2. LASSO REGRESSION
> 3. LARS LASSO REGRESSION
> 4. RIDGE REGRESSION
> 5. BAYESIAN RIDGE

Referência: https://goo.gl/gRYnWW

***

## **A. Importação de Biblioteca Python**

#### Serão utilizadas neste projeto as seguintes bibliotecas Python:

> 1. Pandas
> 2. Numpy
> 3. Scikit Learn
> 4. Seaborn


In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
from sklearn.metrics import r2_score
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import seaborn as sns

# pre-processing
#from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import LabelEncoder

# Cross validation
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import ShuffleSplit
from sklearn.model_selection import cross_val_predict

# Model selection
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold

# Machine learning

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.grid_search import GridSearchCV

# Métrica
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.metrics import make_scorer

# definindo que a execução do código na linha de baixo.
%matplotlib inline

# Ajustando globalmente o impedimento de notação científica
pd.options.display.float_format = '{:20,.2f}'.format

# Variável para definir a amostragem para head() e Tail()
H = 1

/Users/tec/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


***

## **B. Preparação de Dataset**


### B.1 Importando o dataset
Será utilizado o dataset individual criado para o **Dep. Domingos Neto**:

> - df_roc_con.csv


In [2]:
display(os.listdir('dataset'))

['.DS_Store',
 'Ano-2015.csv',
 'Ano-2016.csv',
 'Ano-2017.csv',
 'df_dom_con.csv',
 'df_fco_con.csv',
 'df_roc_con.csv',
 'df_trienio_limpo.csv']

In [3]:
# importando o dataset limpo.
df = pd.read_csv('dataset/df_roc_con.csv', sep = ';', encoding='latin1')

In [4]:
df.Tipo.unique()

array(['Alimentação', 'C&L', 'Consultorias', 'Correios', 'Divulgação',
       'Escritório', 'Hospedagem', 'Locação de Carro', 'Passagens Aéreas',
       'S/D', 'Telefonia', 'Taxi'], dtype=object)

***
## C. DEPUTADO ROCHA

## C.1 CRIANDO X e y

In [5]:
# Criação da variáveis X e y.
X = df
y = X.pop('Valor')

In [6]:
X.head(H)

,Tipo,Ano
0,Alimentação,2015


In [7]:
y.head(H)

0               7,207.45
Name: Valor, dtype: float64

In [8]:
X.shape

(33, 2)

## C.2 CRIANDO UM DATAFRAME PARA PREDIÇÃO DE 2018

Antes de aplicarmos get_dummies na feature Tipo, precisamos listar todo o conteúdo único contido para criarmos o dataframe de predição para 2018.

In [9]:
uniq = X.Tipo.unique()
#display(uniq)
uniq = pd.DataFrame(uniq, columns=['Tipo'])
#display(uniq)
df_2018 = pd.get_dummies(uniq, columns = ['Tipo'])
#display(df_tipo)
df_2018['Ano'] = 2018
#ano = df_2018.pop('Ano')
#df_2018['Ano'] = ano
df_2018.head(12)
#ano = df_2018.pop('Ano')
df_2018 = df_2018.drop('Ano', axis = 1)
df_2018.insert(0, 'Ano', 2018)
df_2018.head(20)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
0,2018,1,0,0,0,0,0,0,0,0,0,0,0
1,2018,0,1,0,0,0,0,0,0,0,0,0,0
2,2018,0,0,1,0,0,0,0,0,0,0,0,0
3,2018,0,0,0,1,0,0,0,0,0,0,0,0
4,2018,0,0,0,0,1,0,0,0,0,0,0,0
5,2018,0,0,0,0,0,1,0,0,0,0,0,0
6,2018,0,0,0,0,0,0,1,0,0,0,0,0
7,2018,0,0,0,0,0,0,0,1,0,0,0,0
8,2018,0,0,0,0,0,0,0,0,1,0,0,0
9,2018,0,0,0,0,0,0,0,0,0,1,0,0


## C.3 TRATANDO FEATURE COM CLASSIFICAÇÃO
Tendo em vista que os algoritmos de machine learning da biblioteca Scikit Learn não tratam automáticamente com colunas com tipo de dados Objeto utilizaremos a função get_dummies do Pandas.
Esta função irá criar colunas adicionais para representar cada tipo de gasto existente e indicará através de 0 e 1 se ela está presente na linha ou não.
Maiores informações podem ser encontradas neste link: 
> - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [10]:
# Vamos utilizar get_dummies para tratar os dados não numéricos da feature 'Tipo'.
X = pd.get_dummies(X, columns = ['Tipo'])

In [11]:
X.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
0,2015,1,0,0,0,0,0,0,0,0,0,0,0


In [12]:
X.shape

(33, 13)

## C.4 'LOGARITMIZAÇÃO'

In [13]:
# Aplicação de logaritmica
    
yl = np.log(y)
Xl = X

## C.5 DATA SPLIT

Como estaremos realizando o uso dos dados de 2015, 2016 e 2017 para realizar a predição dos gastos de 2018 de três deputados, iremos então separar os dados de 2015 e 2016 para serem utilizados como train e os dados de 2017 para test.
O splot de dados será realizado COM e SEM a aplicação de log em y.

In [14]:
# Test split percentual e random state.
per = 0.20
random = None

# data split SEM LOG.
# Data split com 20% de dados para treinamento.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = per, random_state = random)

# data split COM LOG.
# Data split com 20% de dados para treinamento.
Xl_train, Xl_test, yl_train, yl_test = train_test_split(Xl, yl, test_size = per, random_state = random)

In [15]:
X_train.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
21,2016,0,0,0,0,0,0,0,0,0,0,0,1


In [16]:
X_test.head(H)

,Ano,Tipo_Alimentação,Tipo_C&L,Tipo_Consultorias,Tipo_Correios,Tipo_Divulgação,Tipo_Escritório,Tipo_Hospedagem,Tipo_Locação de Carro,Tipo_Passagens Aéreas,Tipo_S/D,Tipo_Taxi,Tipo_Telefonia
8,2015,0,0,0,0,0,0,0,0,1,0,0,0


In [17]:
y_train.head(H)

21              17,552.88
Name: Valor, dtype: float64

In [18]:
yl_train.head(H)

21                   9.77
Name: Valor, dtype: float64

In [19]:
y_test.head(H)

8             173,054.25
Name: Valor, dtype: float64

In [20]:
yl_test.head(H)

26                  12.39
Name: Valor, dtype: float64

## C.6 PREDIÇÃO POR REGRESSÃO LINEAR

### CRIAÇÃO DE FUNÇÃO PARA CONSTRUÇÃO DE TABELA RESULTADO


In [21]:
# Criando tabela com o resultado da regressão
def tabela_resultado(modelo, df):
    pr = modelo.predict(df)
    pr = pd.DataFrame(data=pr)
    #display(pr)
    pr['Valor'] = pr
    #display(pr)
    pr = pr.pop('Valor')
    #display(pr)
    co = df_2018.columns
    co = pd.DataFrame(data=co)
    #display(co)
    co.drop(0, axis = 0, inplace = True)
    co['Tipo'] = co
    co = co.pop('Tipo')
    co = co.reset_index(drop=True)
    #display(co)
    results = pd.concat([co, pr], axis = 1)
    return results
    #display(results)

### CRIAÇÃO DE DATAFRAME PARA PREDIÇÃO PARA 2018

***
## D. LINEAR REGRESSION

### D.1 SEM APLICAÇÃO DE  LOG

In [22]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)
display('Score para train ', lr.score(X_train,y_train).round(4))
display('Score para test ', lr.score(X_test,y_test).round(4))


'Score para train '

0.9659

'Score para test '

0.8468

In [23]:
y_test

8              173,054.25
30             218,674.23
20                 738.28
19              17,471.07
4              165,993.33
14             310,900.00
22                 926.06
Name: Valor, dtype: float64

### D.2 PREDIÇÃO PARA 2018 SEM LOG

In [24]:
# Predição
# Data recover
resultado = tabela_resultado(lr, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"16,184.72"
1,Tipo_C&L,"56,519.80"
2,Tipo_Consultorias,"14,862.71"
3,Tipo_Correios,"11,229.70"
4,Tipo_Divulgação,"245,051.35"
5,Tipo_Escritório,"18,038.36"
6,Tipo_Hospedagem,"18,407.42"
7,Tipo_Locação de Carro,"68,352.24"
8,Tipo_Passagens Aéreas,"208,942.16"
9,Tipo_S/D,"21,037.71"


765375.79

***
### D.3 COM APLICAÇÃO DE  LOG

In [25]:
lrl = linear_model.LinearRegression()
lrl.fit(Xl_train,yl_train)
display('Score para train ', lrl.score(Xl_train,yl_train).round(4))
display('Score para test ', lrl.score(Xl_test,yl_test).round(4))


'Score para train '

0.887

'Score para test '

0.8776

### D.4 PREDIÇÃO PARA 2018 COM LOG

In [26]:
# Predição
#tabela_resultado(lrl, df_2018)
# Data recover
resultado = tabela_resultado(lrl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"4,663.71"
1,Tipo_C&L,"44,999.93"
2,Tipo_Consultorias,"2,506.12"
3,Tipo_Correios,"1,655.12"
4,Tipo_Divulgação,"210,784.23"
5,Tipo_Escritório,"7,870.63"
6,Tipo_Hospedagem,"10,135.18"
7,Tipo_Locação de Carro,"44,701.57"
8,Tipo_Passagens Aéreas,"212,222.09"
9,Tipo_S/D,"13,039.31"


566403.44

***

***
## E. LASSO REGRESSION

***

### E.1 SEM APLICAÇÃO DE LOG

In [27]:
from sklearn import linear_model
la = linear_model.Lasso(alpha = 100)
la.fit(X_train,y_train)
display('Score para train ', la.score(X_train,y_train).round(4))
display('Score para test ', la.score(X_test,y_test).round(4))

'Score para train '

0.9655

'Score para test '

0.8778

### E.2 PREDIÇÃO PARA 2018  SEM LOG

In [28]:
resultado = tabela_resultado(la, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"17,673.40"
1,Tipo_C&L,"55,803.66"
2,Tipo_Consultorias,"16,313.97"
3,Tipo_Correios,"12,248.71"
4,Tipo_Divulgação,"242,525.51"
5,Tipo_Escritório,"19,058.78"
6,Tipo_Hospedagem,"19,428.86"
7,Tipo_Locação de Carro,"67,641.36"
8,Tipo_Passagens Aéreas,"206,498.27"
9,Tipo_S/D,"21,238.40"


721928.04

***

### E.3 COM APLICAÇÃO DE LOG

In [29]:
from sklearn import linear_model
lal = linear_model.Lasso(alpha = 100)
lal.fit(Xl_train,yl_train)
display('Score para train ', lal.score(Xl_train,yl_train).round(4))
display('Score para test ', lal.score(Xl_test,yl_test).round(4))

'Score para train '

0.0

'Score para test '

-0.0378

### E.4 PREDIÇÃO PARA 2018 COM LOG

In [30]:
resultado = tabela_resultado(lal, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"14,871.24"
1,Tipo_C&L,"14,871.24"
2,Tipo_Consultorias,"14,871.24"
3,Tipo_Correios,"14,871.24"
4,Tipo_Divulgação,"14,871.24"
5,Tipo_Escritório,"14,871.24"
6,Tipo_Hospedagem,"14,871.24"
7,Tipo_Locação de Carro,"14,871.24"
8,Tipo_Passagens Aéreas,"14,871.24"
9,Tipo_S/D,"14,871.24"


178454.92

***

***

## F. LARS LASSO

### F.1 SEM APLICAÇÃO DE LOG

In [31]:
from sklearn import linear_model
ll = linear_model.LassoLars(alpha=.1)
ll.fit(X_train,y_train)
display('Score para train ', ll.score(X_train,y_train).round(4))
display('Score para test ', ll.score(X_test,y_test).round(4))

'Score para train '

0.9659

'Score para test '

0.8783

### F.2 PREDIÇÃO PARA 2018 SEM LOG

In [32]:
resultado = tabela_resultado(ll, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"16,185.47"
1,Tipo_C&L,"56,517.58"
2,Tipo_Consultorias,"14,863.66"
3,Tipo_Correios,"11,230.30"
4,Tipo_Divulgação,"245,048.40"
5,Tipo_Escritório,"18,038.96"
6,Tipo_Hospedagem,"18,408.02"
7,Tipo_Locação de Carro,"68,350.02"
8,Tipo_Passagens Aéreas,"208,938.79"
9,Tipo_S/D,"21,035.91"


722619.08

### F.3 COM APLICAÇÃO DE LOG

In [33]:
from sklearn import linear_model
lll = linear_model.LassoLars(alpha=.1)
lll.fit(Xl_train,yl_train)
display('Score para train ', lll.score(Xl_train,yl_train).round(4))
display('Score para test ', lll.score(Xl_test,yl_test).round(4))

'Score para train '

0.2278

'Score para test '

0.1989

### F.4 PREDIÇÃO PARA 2018 COM LOG

In [34]:
resultado = tabela_resultado(lll, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"14,240.68"
1,Tipo_C&L,"14,240.68"
2,Tipo_Consultorias,"14,240.68"
3,Tipo_Correios,"10,279.23"
4,Tipo_Divulgação,"38,836.37"
5,Tipo_Escritório,"14,240.68"
6,Tipo_Hospedagem,"14,240.68"
7,Tipo_Locação de Carro,"14,240.68"
8,Tipo_Passagens Aéreas,"17,082.82"
9,Tipo_S/D,"14,240.68"


189574.43

***

***

## G. RIDGE REGRESSION

### G.1 SEM APLICAÇÃO DE LOG

In [35]:
from sklearn import linear_model
ri = linear_model.Ridge (alpha = .5)
ri.fit(X_train,y_train)
display('Score para train ', ri.score(X_train,y_train).round(4))
display('Score para test ', ri.score(X_test,y_test).round(4))

'Score para train '

0.8827

'Score para test '

0.6978

### G.2 PREDIÇÃO PARA 2018 SEM LOG

In [36]:
resultado = tabela_resultado(ri, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"33,903.68"
1,Tipo_C&L,"64,538.44"
2,Tipo_Consultorias,"31,271.90"
3,Tipo_Correios,"25,718.36"
4,Tipo_Divulgação,"187,799.66"
5,Tipo_Escritório,"31,554.35"
6,Tipo_Hospedagem,"31,870.69"
7,Tipo_Locação de Carro,"74,680.54"
8,Tipo_Passagens Aéreas,"166,350.46"
9,Tipo_S/D,"36,211.90"


785105.45

### G.3 COM APLICAÇÃO DE LOG

In [37]:
from sklearn import linear_model
ril = linear_model.Ridge (alpha = .5)
ril.fit(Xl_train,yl_train)
display('Score para train ', ril.score(Xl_train,yl_train).round(4))
display('Score para test ', ril.score(Xl_test,yl_test).round(4))

'Score para train '

0.8362

'Score para test '

0.8155

### G.4 PREDIÇÃO PARA 2018 COM LOG

In [38]:
resultado = tabela_resultado(ril, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"6,178.03"
1,Tipo_C&L,"40,167.01"
2,Tipo_Consultorias,"3,709.95"
3,Tipo_Correios,"2,662.13"
4,Tipo_Divulgação,"130,297.17"
5,Tipo_Escritório,"9,267.73"
6,Tipo_Hospedagem,"11,197.78"
7,Tipo_Locação de Carro,"39,938.64"
8,Tipo_Passagens Aéreas,"87,024.83"
9,Tipo_S/D,"13,891.95"


360272.38

***

***

## H. BAYESIAN RIDGE

### H.1 SEM APLICAÇÃO DE LOG

In [39]:
br = linear_model.BayesianRidge()
br.fit(X_train,y_train)
display('Score para train ', br.score(X_train,y_train).round(4))
display('Score para test ', br.score(X_test,y_test).round(4))

'Score para train '

0.0

'Score para test '

-0.6528

### H.2 PREDIÇÃO PARA 2018 SEM LOG

In [40]:
resultado = tabela_resultado(br, df_2018)
#resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"35,364.65"
1,Tipo_C&L,"35,364.65"
2,Tipo_Consultorias,"35,364.65"
3,Tipo_Correios,"35,364.65"
4,Tipo_Divulgação,"35,364.65"
5,Tipo_Escritório,"35,364.65"
6,Tipo_Hospedagem,"35,364.65"
7,Tipo_Locação de Carro,"35,364.65"
8,Tipo_Passagens Aéreas,"35,364.65"
9,Tipo_S/D,"35,364.65"


424375.78

***

### H.3 COM APLICAÇÃO DE LOG

In [41]:
brl = linear_model.BayesianRidge()
brl.fit(Xl_train,yl_train)
display('Score para train ', brl.score(Xl_train,yl_train).round(4))
display('Score para test ', brl.score(Xl_test,yl_test).round(4))

'Score para train '

0.8731

'Score para test '

0.8646

### H.4 PREDIÇÃO PARA 2018 COM LOG

In [42]:
resultado = tabela_resultado(brl, df_2018)
resultado['Valor']= np.exp(resultado['Valor'])
display(resultado)
resultado['Valor'].sum().round(2)

,Tipo,Valor
0,Tipo_Alimentação,"5,406.45"
1,Tipo_C&L,"42,910.89"
2,Tipo_Consultorias,"3,063.34"
3,Tipo_Correios,"2,108.37"
4,Tipo_Divulgação,"167,246.61"
5,Tipo_Escritório,"8,585.31"
6,Tipo_Hospedagem,"10,686.34"
7,Tipo_Locação de Carro,"42,645.84"
8,Tipo_Passagens Aéreas,"130,515.77"
9,Tipo_S/D,"13,536.73"


441553.92

Referências:
> 1. https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
> 2.  https://chrisalbon.com/machine_learning/preprocessing_structured_data/preprocessing_categorical_features/
> 3. https://www.packtpub.com/mapt/book/big_data_and_business_intelligence/9781787286382/2/ch02lvl1sec23/working-with-categorical-variables
> 4. http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
> 5. http://scikit-learn.org/stable/auto_examples/plot_cv_predict.html#sphx-glr-auto-examples-plot-cv-predict-py
> 6. http://fastml.com/how-to-use-pd-dot-get-dummies-with-the-test-set/
> 7. Scikit-learn Cookbook - Second Edition - Julian Avila, Trent Hauck - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781787286382
> 8. scikit-learn : Machine Learning Simplified - Raúl Garreta, Guillermo Moncecchi, Trent Hauck, Gavin Hackeling - November 2017 - https://www.packtpub.com/mapt/book/all_books/9781788833479
> 9. Mastering Machine Learning with scikit-learn - Second Edition - Gavin Hackeling - July 2017 - https://www.packtpub.com/mapt/book/all_books/9781788299879
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 
> 